In [1]:
##########################################################################
#### Python Code demonstrates benefit of stacking the raw data,       ####
#### supplied by Louis Kyphen, in retaining most of the observations, ####
#### rather than removing mostly empty columns. Restructering data    ####
#### also facilitates focus on specific sampling tests.               #### 
#### THe code also shows use of Data Dictionary test ranges,          ####
#### for purposes of data quality control.                            ####
##########################################################################

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
%matplotlib inline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE


#############################################################################
##### Following function uploads the four data files provided by Louis Kyphen,
##### and joins them on the field EventId.
##### User of this code assigns a string to the DFPath string variable,
##### make sure the data file names are the original names and files are csv.
##### Note all lines outputing csv files are commented out throughout the code.
#############################################################################

DFPath = 'C:/Users/bobby/OneDrive/Desktop/'

file_1 = pd.read_csv(DFPath + 'Chemical_DataPull_12272022.csv')
file_2 = pd.read_csv(DFPath + 'Bacterial_DataPull_12272022.csv')
file_3 = pd.read_csv(DFPath + 'StreamHabitat_DataPull_12272022.csv')
file_4 = pd.read_csv(DFPath + 'Macroinverterbrate_DataPull_12272022.csv')


def JoinData(DFPath,file_1,file_2,file_3,file_4):
    dataset_left = file_1
    dataset_right = file_2
    dataset_left.merge(file_2, on= ['EventID','EventDate', 'GroupID','GroupName','site_latitude',\
    'site_longitude','site_altitude','SiteID','SiteName'], how='outer')
    dataset_right = file_3 
    dataset_left.merge(dataset_right, on= ['EventID','EventDate', 'GroupID','GroupName','site_latitude',\
    'site_longitude','site_altitude','SiteID','SiteName'], how='outer')
    dataset_right = file_4
    data_out = dataset_left.merge(dataset_right, on= ['EventID','EventDate', 'GroupID','GroupName','site_latitude',\
    'site_longitude','site_altitude','SiteID','SiteName'], how='outer')
    return(data_out)


data_all = JoinData(DFPath,file_1,file_2,file_3,file_4)
data_all.head()
data_all.describe()
data_all.shape

##### data_all.to_csv(DFPath + 'data_all3.csv')


##################################################################
##### Prepping the data, keep most of the data, rather than discarding it
##### Will remove all columns with percent missing = .9995 columns have least 36 values.
##### For remaining columns, this provides a reasonable minimum sample count in each column.
##################################################################


data_all.isnull().any().any()
x = data_all.isnull().sum() 
y = x[x > 0]
null_percent = y/data_all.shape[0]
null_percent


missing_features = null_percent[null_percent > 0.9995].index
data_all.drop(missing_features, axis=1, inplace=True)


print('Dropped columns:', missing_features)


#### next line of code outputs data 
#### data_all.to_csv(DFPath + 'df_clean.csv')

data_all.shape
data_all.describe()
data_all.head()

#######################################################################################
#### Stacking Sample Data serves to separate individual sample tests.
#### Following code stacks the data using Python melt method, 
#### Other methods such as stack and pivot accomplish similar data restructuring.
#### id_vars variables appear on the each row of the stacked data, and all other columns get stacked.
#### Of the 72,261 unique EventIDs,from the combined data file,
#### stacking and deleting rows with no values keeps 71,474 unique eventIDs.
#### Output file for stacking has about 500K rows.
######################################################################################

df_melt = data_all
df_melt = df_melt.melt(id_vars =["EventID", "EventDate", "GroupID", "GroupName", "SiteID", "SiteName", \
        "site_latitude", "site_longitude", "site_altitude"]).dropna().reset_index()

df_melt.shape

df_melt.head(10)

df_melt.describe()

### df_melt.to_csv(DFPath + 'df_melt.csv')

df_melt.groupby('EventID').nunique()
data_all.groupby('EventID').nunique()


##########################################################################################
##### Many of the values are string type, especially for the Macrointerverbrate measurement columns.
##### Additionally, the data has multiple comments, which are string values
##### It is sensible to break the full data file into two files, one for the numeric values, and another,
##### for the categorical values. Remaining code deals with only numerical values, 
##### and using the Data Dictionary to classify the values as plausible and/or typical.
##### Output file has new 2 binary columns, showing the value column (the measurement) 
##### as plausible and/or typical. Note range values in the g(x) can be changed easily.
##########################################################################################

df_num = df_melt[pd.to_numeric(df_melt.value, errors= 'coerce').notnull()]

### df_num.to_csv(DFPath + 'df_num.csv')


df_num['value'] = pd.to_numeric(df_num['value'], errors='coerce')

def f(x):
  if x["value"] < 0: return 0
  else: return 1

df_num["plausible_values"] = df_num.apply(f, axis=1)


def g(x):
  if x["variable"] == "DissolvedOxygen1" and 5 <= x["value"]<= 12 \
  or x["variable"] == "DissolvedOxygen2" and 5 <= x["value"]<= 12 \
  or x["variable"] == "ph1" and 0 <= x["value"]<= 14 \
  or x["variable"] == "ph2" and 0 <= x["value"]<= 14 \
  or x["variable"] == "Conductivity" and 0 <= x["value"]<= 10000 \
  or x["variable"] == "SecchiDisk1" and 0 <= x["value"]<= 10 \
  or x["variable"] == "SecchiDisk2" and 0 <= x["value"]<= 10 \
  or x["variable"] == "do_saturation" and 80 <= x["value"]<= 120 \
  or x["variable"] == "Turbidity" and 0 <= x["value"]<= 1000 \
  or x["variable"] == "Salinity1" and 0 <= x["value"]<= 35 \
  or x["variable"] == "Salinity2" and 0 <= x["value"]<= 35 \
  or x["variable"] == "Alkaliniy" and 20 <= x["value"]<= 200 \
  or x["variable"] == "Orthophosphate" and 1 <= x["value"]<= 3.5 \
  or x["variable"] == "NitrateN" and 0 <= x["value"]<= 10 \
  : return 1
  else: return 0

df_num["typical_values"] = df_num.apply(g, axis=1)

##### must delete any empty rows appearing in output csv file

df_num.replace('', np.nan, inplace=True)
df_num.dropna(inplace=True)
#### df_num.to_csv(DFPath + 'df_numrev2.csv')

df_num.shape
df_num.describe()
df_num.head()


####################################################################################
##### Basic chart shows typical(1)/nontypical(0) and plausible (1)/ not plausible(0)
##### Most values are plausible, however many are not typical, 
##### based on data dictionary ranges shown in the Python code.
###################################################################################

sns.countplot(df_num['plausible_values'])
plt.show()

sns.countplot(df_num['typical_values'])
plt.show()


#### select all values which are plausible and continue with further work in
#### basic data analysis

df_pl = df_num[df_num['plausible_values'] > 0]
df_pl.head()


plt.figure(figsize=(15,8))
chart = df_pl['variable'].value_counts().plot(kind='bar')
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')



##############################################################################################
####  Following function adds two columns useful for attribute analysis:
####  a count for siteName attached to each row, as well as a count of observations for each type 
####  chemical or biological test recorded
##############################################################################################


def AddCountVar(df_pl,countvar1,namecount):
    newdata1 = df_pl.groupby([countvar1]).size().sort_values(ascending=False).reset_index(name = namecount)
    newdata2 = pd.merge(df_pl, newdata1)
    return(newdata2)

df_obs = AddCountVar(df_pl,'variable','countObs')

df_obs2 = AddCountVar(df_obs,'SiteName','countSites')


####################################################################################
#### df_obs2 is the original data observations with added columns for
#### typical values, plausible values, count of observations by type of sampling done
#### and count of observations made at each SiteName
####################################################################################

#### df_obs2.to_csv(DFPath + 'df_obs2.csv')


######################################################################################
##### Draw histogram showing SiteName Counts to allow subsetting the data. 
##### Task here is to chart SiteNames with highest counts of observations.
##### Bins provide amount detail to allow user to cut the data.
######################################################################################


plt.hist(df_obs2['countSites'],color = 'blue', edgecolor = 'black', bins = 30)

plt.show()

df_obs3 = df_obs2[df_obs2['countSites'] > 2000]

##### Here, we select all SiteNames with a total of 2000 observations to chart

df_obs3.shape

chart = df_obs3['SiteName'].value_counts().plot(kind='bar')
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

########################################################################################
##### As a final task in the notebook, we draw chart box plots of 
##### DissolvedOxygen1 to show which sites may be outliers for mean measure
##### More extensive data analysis using ANOVA methods can be used for data quality control
##### In future notebooks.
########################################################################################



df_obs4 = df_obs3[df_obs3['variable'] == 'DissolvedOxygen1']

plt.figure(figsize=(15,8))
ax = sns.boxplot(x='SiteName', y='value', data=df_obs4, color='#99c2a2')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45);

########################################################################################
### SiteName Boot Harbor appears as outlier for DissolvedOxygen1 test, several others appear
### More definitive statistics tests including defined outlier SiteNames can be confirmed with ANOVA.
### ####################################################################################


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/bobby/OneDrive/Desktop/Chemical_DataPull_12272022.csv'